# Weapon Detection System Using YOLOv8
## Real-Time Weapon Detection with Transfer Learning

## 1. Setup and Installation

In [ ]:
#!pip install -U "numpy<2" ultralytics opencv-python pillow matplotlib pandas


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
from pathlib import Path
import os

## 2. Dataset Configuration

In [ ]:
dataset_config = """
path: /kaggle/input/weapon-detecttion-dataset-roboflow
train: train/images
val: valid/images

nc: 5
names: ['Grenade', 'Knife', 'Missile', 'Pistol', 'Rifle']
"""

with open('weapon_data.yaml', 'w') as f:
    f.write(dataset_config)

print("Dataset configuration created successfully")

In [ ]:
class_names = ['Grenade', 'Knife', 'Missile', 'Pistol', 'Rifle']
num_classes = len(class_names)

print(f"Total Classes: {num_classes}")
print(f"Class Names: {class_names}")

## 3. Model Training with Transfer Learning

In [ ]:
model = YOLO('yolov8n.pt')

print("YOLOv8 Nano model loaded with COCO pre-trained weights")
print("Transfer learning: Fine-tuning entire model for weapon detection")

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data='/kaggle/working/weapon_data.yaml' epochs=50 imgsz=640

## 4. Training Results Visualization

In [ ]:
results_path = Path('runs/detect/train')

training_results = Image.open(results_path / 'results.png')
plt.figure(figsize=(16, 10))
plt.imshow(training_results)
plt.axis('off')
plt.title('Training Metrics Over Epochs', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
confusion_matrix = Image.open(results_path / 'confusion_matrix.png')
plt.figure(figsize=(12, 10))
plt.imshow(confusion_matrix)
plt.axis('off')
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Model Evaluation on Validation Set

In [ ]:
trained_model = YOLO(results_path / 'weights/best.pt')

print("Best model loaded for evaluation")

In [ ]:
metrics = trained_model.val(data='weapon_data.yaml')

print("\n" + "="*60)
print("MODEL PERFORMANCE METRICS")
print("="*60)
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")
print("="*60)

In [ ]:
  available_classes = len(metrics.box.p)
  per_class_metrics = pd.DataFrame({
      'Class': class_names[:available_classes],
      'Precision': metrics.box.p,
      'Recall': metrics.box.r,
      'mAP50': metrics.box.ap50,
      'mAP50-95': metrics.box.ap
  })

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

per_class_metrics.plot(x='Class', y='Precision', kind='bar', ax=axes[0,0], color='steelblue', legend=False)
axes[0,0].set_title('Precision by Class', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('Precision')
axes[0,0].tick_params(axis='x', rotation=45)

per_class_metrics.plot(x='Class', y='Recall', kind='bar', ax=axes[0,1], color='coral', legend=False)
axes[0,1].set_title('Recall by Class', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('Recall')
axes[0,1].tick_params(axis='x', rotation=45)

per_class_metrics.plot(x='Class', y='mAP50', kind='bar', ax=axes[1,0], color='green', legend=False)
axes[1,0].set_title('mAP50 by Class', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('mAP50')
axes[1,0].tick_params(axis='x', rotation=45)

per_class_metrics.plot(x='Class', y='mAP50-95', kind='bar', ax=axes[1,1], color='purple', legend=False)
axes[1,1].set_title('mAP50-95 by Class', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('mAP50-95')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Prediction on Test Images

In [ ]:
test_images_path = Path('/kaggle/input/weapon-detecttion-dataset-roboflow/test/images')
test_images = list(test_images_path.glob('*.jpg')) + list(test_images_path.glob('*.jpeg')) + list(test_images_path.glob('*.png'))

print(f"Found {len(test_images)} test images")

In [ ]:
def predict_and_visualize(image_path, model, conf_threshold=0.25):
    results = model.predict(source=image_path, conf=conf_threshold, save=False)
    
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            label = f"{class_names[cls]} {conf:.2f}"
            
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Detection: {image_path.name}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return results

In [ ]:
for test_img in test_images[120:150]:
    prediction_results = predict_and_visualize(test_img, trained_model, conf_threshold=0.3)

## 7. Batch Prediction and Statistics

In [ ]:
all_detections = []

for img_path in test_images:
    results = trained_model.predict(source=img_path, conf=0.25, save=False, verbose=False)
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            all_detections.append({
                'image': img_path.name,
                'class': class_names[int(box.cls[0])],
                'confidence': float(box.conf[0]),
                'bbox': box.xyxy[0].tolist()
            })

detections_df = pd.DataFrame(all_detections)
print(f"\nTotal detections: {len(detections_df)}")
print(f"\nDetections by class:")
print(detections_df['class'].value_counts())

In [ ]:
if len(detections_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    detections_df['class'].value_counts().plot(kind='bar', ax=axes[0], color='teal')
    axes[0].set_title('Detections by Weapon Class', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Weapon Type')
    axes[0].set_ylabel('Count')
    axes[0].tick_params(axis='x', rotation=45)
    
    axes[1].hist(detections_df['confidence'], bins=20, color='orange', edgecolor='black')
    axes[1].set_title('Confidence Score Distribution', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Confidence')
    axes[1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

## 8. Video Prediction for Real-Time Detection

In [ ]:
def predict_video(video_path, model, output_path='output_video.mp4', conf_threshold=0.3):
    cap = cv2.VideoCapture(video_path)
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = 0
    detection_count = 0
    
    print(f"Processing video: {total_frames} frames at {fps} FPS")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model.predict(source=frame, conf=conf_threshold, save=False, verbose=False)
        
        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                label = f"{class_names[cls]} {conf:.2f}"
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                detection_count += 1
        
        out.write(frame)
        frame_count += 1
        
        if frame_count % 30 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")
    
    cap.release()
    out.release()
    
    print(f"\nVideo processing complete!")
    print(f"Total frames processed: {frame_count}")
    print(f"Total detections: {detection_count}")
    print(f"Output saved to: {output_path}")

In [ ]:
video_path = 'test_video.mp4'

if os.path.exists(video_path):
    predict_video(video_path, trained_model, output_path='weapon_detection_output.mp4', conf_threshold=0.3)
else:
    print(f"Video file not found: {video_path}")
    print("Place your test video in the project directory and update the path")

## 9. Real-Time Webcam Detection

In [ ]:
def realtime_webcam_detection(model, conf_threshold=0.3):
    cap = cv2.VideoCapture(0)
    
    print("Starting webcam detection...")
    print("Press 'q' to quit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model.predict(source=frame, conf=conf_threshold, save=False, verbose=False)
        
        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                label = f"{class_names[cls]} {conf:.2f}"
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
        cv2.imshow('Weapon Detection - Press Q to Quit', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Webcam detection stopped")

In [ ]:
realtime_webcam_detection(trained_model, conf_threshold=0.3)

## 10. Model Export for Deployment

In [ ]:
trained_model.export(format='onnx')
print("Model exported to ONNX format for deployment")

In [ ]:
model_info = {
    'Model': 'YOLOv8n',
    'Classes': num_classes,
    'Class Names': ', '.join(class_names),
    'Transfer Learning': 'COCO pre-trained weights',
    'Fine-tuning': 'Full model (unfrozen)',
    'Image Size': '640x640',
    'Epochs': 50,
    'Batch Size': 16,
    'Optimizer': 'Adam',
    'Best Weights': str(results_path / 'weights/best.pt')
}

print("\n" + "="*60)
print("MODEL CONFIGURATION SUMMARY")
print("="*60)
for key, value in model_info.items():
    print(f"{key}: {value}")
print("="*60)